# Reconstruct even/odd Tomograms

Now that we have split the data into an even and odd part, we need to construct a tomogram from these images. For this we will use IMOD. We will do reconstruction on the 'full' frames as they have the best signal-to-noise ratio and then copy the reconstruction parameters to automatically reconstruct the same tomograms from the even and odd half-datasets. First we need to make the separate frames into a stack, so run the code below and follow instructions

In [ ]:
from shutil import copy as cp

from os.path import join, exists

from utils import modify_com_scripts, reconstruct_tomo

import mrcfile
from matplotlib import pyplot as plt
import sys

import os
from glob import glob
import subprocess
import re #regular expressions (regex)

In [ ]:
# imports and settings for the GPU

os.environ["CUDA_VISIBLE_DEVICES"] = "0" # e.g. "0,1"

In [ ]:
# Path to the parent directory of the even/odd splits
data_path = 'frames/'

#set up some directories
if not os.path.isdir(join(data_path, 'even/tomogram')):
    os.mkdir(join(data_path, 'even/tomogram'))
if not os.path.isdir(join(data_path, 'odd/tomogram')):
    os.mkdir(join(data_path, 'odd/tomogram'))
if not os.path.isdir(join(data_path, 'full/tomogram')):
    os.mkdir(join(data_path, 'full/tomogram'))

Now we turn the .mrc files into a nicely aligned stack, based on the filenames. If this doesn't work, it may be because filenames are not in the standard formatting. You can also manually turn the files into a stack if you can't figure out how to fix this. Just make sure the stack ends up in the /tomogram folder of even, odd, and full.

In [ ]:
directories = ['even','odd','full']
for directory in directories:    
    # get the raw files
    file_list = glob(join(data_path , directory , '*.mrc')) #change to mrc if mrc
    
    # sort the filenames as bidirectional for easy inspection
    pattern = re.compile(r'(-?\d+.?\d+).mrc') # we set up a regex pattern (jeol3200 format), adjust if needed
    def tilt_sorter(x):
        return(int(float((re.findall(pattern, x)[0]))))
    sorted_file_list = sorted(file_list, key = tilt_sorter) 
    
    # now make the sorted list into a stack storedin the /tomogram directories
    cmd = ['newstack'] + sorted_file_list + [data_path + directory + '/tomogram/' + 'tomostack.st']
    #print(" ".join(cmd))
    subprocess.run(" ".join(cmd), shell = True)

remove original .tif files from /frames as we have the full aligned frames in /frames/full so these files are no longer needed. This is to save space on the cluster. The default is to keep them, but it would be good practice to set the statement to True below, and hereby remove these now unnecessary files.

In [ ]:
remove_original_tifs = False # set to true to remove .tifs in the /frames directory (but not in even, odd, full)

if remove_original_tifs:
    !\rm frames/*.tif

Now you should have a .st stack in the /tomogram folder of even, odd and full.
## Tomogram reconstruction
Now reconstruct a tomogram in the `/frames/full/tomogram` folder by navigating to this folder using terminal and then typing `etomo`. If bead clicking is required, copy the stack in `/frames/full/tomogram` to a new folder on your local device and reconstruct there. Copy the contents of the folder (minus the files bigger than 100MB for speed) back to `/frames/full/tomogram`. 
Copying files can be done via `rsync` or via filezilla. The latter is the easiest.

(note:  we dont need files such as .rec, .ali, .st, so copying them back to hpc04 is just a waste of time. Easiest is just to sort by filesize and copy the files < 100MB via filezilla)

## Copy Reconstruction Files

In [ ]:
# Path to the imod reconstruction files (i.e. the directory containing full files)
path_recon_files = 'frames/full/tomogram'

In [ ]:
def copy_etomo_files(src, name, target):
    # required for creating final aligned stack
    cp(join(src, 'newst.com'), target)
    cp(join(src, 'rotation.xf'), target)
    cp(join(src, name + '.xf'), target)
    # required for creating tomogram 
    cp(join(src, 'tilt.com'), target)
    cp(join(src, name + '.xtilt'), target)
    cp(join(src, name + '.tlt'), target)
    # files for SIRT (optional, might not exist if user didnt use SIRT)
    if exists(join(src, 'sirtsetup.com')):
        cp(join(src, 'sirtsetup.com'),target);


In [ ]:
# We copy the reconstruction files into the two sub-directories containing
# the even/odd data.
copy_etomo_files(path_recon_files, 'tomostack', data_path + 'even/tomogram')
copy_etomo_files(path_recon_files, 'tomostack', data_path + 'odd/tomogram')

## Reconstruct Tomograms
Specify whether you want to use the SIRT method for tomogram reconstruction. Set to true if you used this in your 'full' reconstruction. 

In [ ]:
# This will write a reconstruction log 
reconstruct_tomo(join(data_path, 'even/tomogram'), 'tomostack', SIRT=False) # final output name will be 'half-tomo.rec'

In [ ]:
# This will write a reconstruction log
reconstruct_tomo(join(data_path, 'odd/tomogram'), 'tomostack', SIRT=False) # final output name will be 'half-tomo.rec'

##  (Optional) Tomogram Inspection

In [ ]:
tomogram_slice_nr = 100 # just the slice number you want to see of the tomogram
with mrcfile.open(join(data_path, 'even/tomogram', 'half-tomo.rec')) as mrc:
    even = mrc.data[tomogram_slice_nr]
    
with mrcfile.open(join(data_path, 'odd/tomogram', 'half-tomo.rec')) as mrc:
    odd = mrc.data[tomogram_slice_nr]

In [ ]:
plt.figure(figsize=(20,10))
plt.subplot(1,2,1)
plt.imshow(even[:200,:200], cmap='gray')
plt.title('half-tomo_even.rec, slice ' + str(tomogram_slice_nr));
plt.subplot(1,2,2)
plt.imshow(odd[:200,:200], cmap='gray')
plt.title('half-tomo_odd.rec, slice ' + str(tomogram_slice_nr));